# 일급 함수 디자인 패턴

- 디자인 패턴이 언어에 독립적이긴 하지만, 모든 언어에 적용될 수 있는것은 아니다.
- 언어는 개발자의 관점에 영향을 미치므로 언어의 선택은 중요.
- 일급 함수를 지원하는 언어제서는 `전략, 명령, 템플릿 메서드, 비지터 패턴`을 다시 생각하라. "피터 노빅"
- 일부 클래스의 객체를 간단한 함수로 교체하면, 반복되는 코드의 상당 부분을 줄일 수 있다.

### `함수객체를 이용해서 전략패턴을 리팩토링하고, 명령 패턴을 단수화하는 방법을 설명`

> 템플릿 메서드 패턴???<br/>
> 여러 클래스에서 공통으로 사용하는 메서드를 상위 클래스에서 정의하고, 하위 클래스마다 다르게 구현해야 하는 세부적인 사항을 하위 클래스에서 구현하는 패턴.

> 비지터 패턴???<br/>
> 객체의 구조와 기능을 분리시키는 패턴. 구조는 변하지 않으며 기능만 따로 추가되거나 확장되어야 할 경우 사용되는 패턴

> 리팩토링???<br/>
> 결과의 변경 없이 코드의 구조를 재조정함. 버그를 없애거나 새로운 기능을 추가하는 행위는 아님.

## 6.1 사례: 전략 패턴의 리팩토링

- 전략 패턴은 파이썬에서 함수를 일급 객체로 사용하면 더욱 간단해질 수 있는 디자인 패턴의 대표적인 사례.

## 6.1.1 고전적인 전략

- Context
    - 일부 계산을 서로 다른 알고리즘을 구현하는 `교환 가능한 컴포넌트에 위임`함으로써 서비스를 제공.
    - ex> 여러 알고리즘 중 하나에 따라 프로모션 할인을 적용하도록 설정.


- Stratagy
    - 여러 알고리즘을 구현하는 컴포넌트에 `공통된 인터페이스`.
    - ex> Promotion 이라는 추상 클래스.


- concrete strategies
    - 전략의 구상 `서브클래스` 중 하나.
    - ex> 3개의 promotion.


- 그림 6-1. 주문 할인 처리에 대한 UML 클래스 다이어그램
<img src='image/fig_6-1.JPG'>

> UML???<br/>
> Unified Modeling Language. 설계도를 그리기 위한 언어. 건축 도면과 유사하게 정해진 기호로 구조를 설명할 수 있도록 하는 언어.

- "디자인 패턴"에서 `전략 패턴` 설명
    - 일련의 알고리즘을 정의하고 각각을 하나의 클래스 안에 넣어서 교체하기 쉽게 만든다.
    - 전략을 이용하면 사용하는 클라이언트에 따라 알고리즘을 독립적으로 변경할 수 있다.
    
- 전자 상거래 영역에서 쉽게 볼 수 있다.
- ex> 온라인 상점의 할인규칙
    1. 충성도 포인트가 1,000점 이상인 고객은 전체 주문에 5% 할인
    2. 하나의 주문에서 20개 이상의 동일 상품을 구입하면 해당 상품에 10% 할인
    3. 서로 다른 상품을 10종류 이상 주문하면 전체 주문 7% 할인


- ex 6-1. 위 온라인 상점의 할인 규칙 적용. 주문 객체를 생성하기 전에 시스템이 할인 전략을 선택해서 Order 생성자에 전달.

> 전략의 선택은 패턴 범위를 벗어난다.???<br/>
> 3개의 할인 방법 중 어느 것을 선택할 것이지는 패턴의 범위를 벗어난 것이다?


> 추상클래스???<br/>
> 추상 클래스는 상속을 강제하기 위한 것. 부모 클래스에는 메소드의 시그니처만 정의해놓고 그 메소드의 실제 동작 방법은 이 메소드를 상속받은 하위 클래스의 책임으로 위임.<br/>
> 기능을 위한 것이 아니라, 상속을 위해서만 존재하는 클래스.

In [2]:
# ex 6-1.
# @abstractmethod 데커레이터를 사용할 수 있도록 Promotion을 추상 베이스 클래스로 구현했으므로, 전략 패턴을 더욱 명시적으로 보여준다.

from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:  # the Context

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


class Promotion(ABC):  # the Strategy: an Abstract Base Class

    @abstractmethod
    def discount(self, order):
        """Return discount as a positive dollar amount"""


class FidelityPromo(Promotion):  # first Concrete Strategy
    """5% discount for customers with 1000 or more fidelity points"""

    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0


class BulkItemPromo(Promotion):  # second Concrete Strategy
    """10% discount for each LineItem with 20 or more units"""

    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount


class LargeOrderPromo(Promotion):  # third Concrete Strategy
    """7% discount for orders with 10 or more distinct items"""

    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

#### ex 6-2 시작

In [3]:
joe = Customer('John Doe', 0)         # <1> 고객 정보 생성
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),    # <2> 쇼핑카트에 항목 3개 객체 생성
         LineItem('apple', 10, 1.5),
         LineItem('watermellon', 5, 5.0)]
Order(joe, cart, FidelityPromo())     # <3> joe의 주문, 충성도 부족으로 충성도 프로모션 불가

<Order total: 42.00 due: 42.00>

In [4]:
Order(ann, cart, FidelityPromo())     # <4> ann은 노예라서 충성도 프로모션 가능

<Order total: 42.00 due: 39.90>

In [5]:
banana_cart = [LineItem('banana', 30, .5),  # <5> 바나나 카트
                LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, BulkItemPromo())    # <6> 바나나가 20개 이상이라 벌크 프로모션 가능

<Order total: 30.00 due: 28.50>

In [6]:
long_order = [LineItem(str(item_code), 1, 1.0)   # <7> 항목이 10개인 카트
               for item_code in range(10)]
Order(joe, long_order, LargeOrderPromo())   # <8> 라지오더프로모션이 가능

<Order total: 10.00 due: 9.30>

In [7]:
Order(joe, cart, LargeOrderPromo())   # 처음 카트에 담긴 항목으로는 받지 못함.

<Order total: 42.00 due: 42.00>

- 둘이 공동구매하면 joe 개이득

#### ex 6-2 종료

## 6.1.2 함수지향 전략

- `ex 6-1`에서 구체적인 전략은 discount()라는 메서드 하나를 가진 클래스.
- 전략 객체(Promotion)는 객체 속성을 가지고 있지 않음.
- `ex 6-3` : `ex 6-1`을 리팩토링. 구체적인 전략을 간단히 함수로 변경하고 `Promotion`추상 클래스를 제거.

In [8]:
# ex 6-3

from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:  # the Context

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)     # <1> 할인액을 계산할 때 self.promotion() 함수를 호출
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

# <2> 추상 클래스 삭제

# <3> 구체적인 전략이 함수로 구현
def fidelity_promo(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0


def bulk_item_promo(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount


def large_order_promo(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

#### ex 6-4 시작

In [9]:
joe = Customer('John Doe', 0)           # <1> 고객 정보 생성
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]
Order(joe, cart, fidelity_promo)        # <2> 할인 함수를 인수로 전달
# Order(joe, cart, FidelityPromo())  <-- ex 6-2

<Order total: 42.00 due: 42.00>

In [10]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [11]:
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [12]:
long_order = [LineItem(str(item_code), 1, 1.0)
              for item_code in range(10)]
Order(joe, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [13]:
Order(joe, cart, large_order_promo)

<Order total: 42.00 due: 42.00>

#### ex 6-4 종료

- `ex 6-4`에서는 Order 객체마다 할인 전략 객체를 만들 필요가 없다. 함수를 바로 사용.
- "디자인 패턴"에서 "전략 객체는 종종 훌륭한 플라이웨이트(flyweight)가 되기도 한다" 라고 함.
- "플라이웨이트는 여러 콘텍스트에서 동시에 사용할 수 있는 공유 객체"라고 정의
- 동일한 전략 객체를 반복해서 적용할 때는 새로 생성하는 비용을 줄이기 위해 `플라이웨이트를 공유`하는 것이 좋다.
- 예제의 경우 `Order(콘텍스트)` 객체를 만들 때 기존 전략 객체가 있으면 재사용 할 수 있다.
- 전략 패턴의 단점인 '런타임 비용'을 극복하기 위해 `프라이웨이트 패턴을 사용`하도록 권고.
- 반면 `소스 코드의 행 수와 유지보수 비용`이 눈덩이처럼 불어난다.???



- 구체적인 전략 객체가 내부 상태를 가지고 있어서 더욱 복잡한 경우, 모든 `전략 패턴과 플라이웨이트 패턴을 혼합`해서 사용.
- 콘텍스트에서 오는 데이터를 처리하는 경우, 추상 클래스에서 정의된 메서드를 하나만 구현하는 클래스를 만들기 보다 `일반 함수를 만드는 것`이 좋다.
    - 함수는 사용자 정의 클래스보다 훨씬 가볍고 파이썬이 모듈을 컴파일할 때 단 한번만 생성되므로 플라이웨이트가 필요하지 않다.
    - 일반 함수도 여러 콘텍스트에서 동시에 공유할 수 있는 `공유객체`


> 플라이웨이트 패턴???<br/>
> 데이터를 공유 사용하여 메모리를 절약할 수 있는 패턴. <br/>
> 일반적으로 공통으로 사용되는 객체는 새로 생성해서 사용하지 않고 공유를 통해 효율적으로 활용.<br/>
> 한 번 생성된 객체는 두 번 생성되지 않고, 풀에 의해서 관리 및 사용된다. 가장 많이 사용되는 패턴 중 하나.<br/>
> https://mrw0119.tistory.com/71

- 가장 좋은 할인 전략을 선택하는 '메타 전략'을 만든다고 가정.
- 코드를 더 리팩토링하고 함수와 모듈을 객체로 활용하는 다양한 방법을 이용해서 최선의 전략을 찾아본다

## 6.1.3 최선의 전략 선택하기 : 단순한 접근법

- ex 6-4와 동일한 고객, 쇼핑 카트가 주어졌을 때 테스트 3개 추가

In [14]:
# ex 6-6 best_promo() 함수

promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    """최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)

#### ex 6-5 시작

In [15]:
# ex 6-5

Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [16]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [17]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

#### ex 6-5 종료

- ex 6-6
    1. 함수로 구현된 리스트 생성
    2. 다른 프로모션 함수들처럼 Order를 객체로 받는다.
    3. 제너레이터 표현식을 이용해서 최대 할인액 계산.
    
    
- 함수가 일급 객체라는 개념에 익숙해 지면 데이터 구조체를 만든다는 생각이 자연스럽게 떠오른다.
- 가독성이 좋으나, 일부 코드가 중복되어 있어 버그가 생길 여지가 있다.
> 어디가 중복일까???

- 새로운 할인 전략 시 문제점
    1. 함수를 코딩하고 이 함수를 promos 리스트에 추가해야 한다.
    2. 새 할인 함수를 Order 객체에 인수로 전달해서 작동시킬 수 있지만, 이 때 best_promo()는 새로운 할인 함수를 고려하지 않는다

## 6.1.4 모듈에서 전략 찾기

- 파이썬 `모듈도 일급 객체`
- 모듈을 다루는 여러 함수가 표준 라이브러리에서 제공.
- globals()
    - 전역 `심벌 테이블`을 나타내는 `딕셔너리 객체` 반환.
    - 이 딕셔너리는 언제나 현재 모듈에 대한 내용을 담고 있다.
    - 함수나 메서드 안에서 호출할 때, 함수를 호출한 모듈이 아니라 `함수가 정의된 모듈`을 나타낸다.
    
- ex 6-7. globals()에 약간 꼼수를 부려 best_promo()가 자동으로 다른 *_promo()함수를 찾게 한다.

> 심볼 테이블???<br/>
> 파이썬에서 모든 데이터는 객체의 형태로 저장.<br/>
> 변수는 객체의 이름일 뿐인데, 이를 심볼이라고 한다.<br/>
> 변수의 이름과 데이터의 주소를 저장하는 테이블.<br/>
> https://victorydntmd.tistory.com/241

In [1]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['', 'globals()'],
 '_oh': {},
 '_dh': ['D:\\study\\Fluent'],
 'In': ['', 'globals()'],
 'Out': {},
 'get_ipython': <bound method InteractiveShell.get_ipython of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x0000024477BB9C08>>,
 'exit': <IPython.core.autocall.ZMQExitAutocall at 0x24477bf1588>,
 'quit': <IPython.core.autocall.ZMQExitAutocall at 0x24477bf1588>,
 '_': '',
 '__': '',
 '___': '',
 '_i': '',
 '_ii': '',
 '_iii': '',
 '_i1': 'globals()'}

In [2]:
locals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['', 'local()', 'locals()'],
 '_oh': {},
 '_dh': ['D:\\study\\Fluent'],
 'In': ['', 'local()', 'locals()'],
 'Out': {},
 'get_ipython': <bound method InteractiveShell.get_ipython of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x0000016F7E895A48>>,
 'exit': <IPython.core.autocall.ZMQExitAutocall at 0x16f7fbb1908>,
 'quit': <IPython.core.autocall.ZMQExitAutocall at 0x16f7fbb1908>,
 '_': '',
 '__': '',
 '___': '',
 '_i': 'local()',
 '_ii': '',
 '_iii': '',
 '_i1': 'local()',
 '_i2': 'locals()'}

In [18]:
# ex 6-7

promos = [globals()[name] for name in globals()     # <1> globals() 함수가 반환한 딕셔너리에서 name을 반복한다.
             if name.endswith('_promo')             # <2> _promo로 끝나는 name만
             and name != 'best_promo']              # <3> 무한 재귀 호출을 피하기위해 best_promo 자신은 뺀다.

def best_promo(order):
    """최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)   # <4> best_promo() 내부는 변경되지 않는다.

In [21]:
promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]

- 별도의 모듈을 만들고, 그 모듈에 넣어서 적용할 수 있는 모든 할인 함수를 모으는 방법도 있다.
- ex 6-8. `promotions`라는 별도 모듈에 내부 조사를 수행해서 만든 전략 함수 리스트를 하용하도록 변경.
- `inspect와 promosions`를 임포트해야 작동.
- inspect : 상위 수준의 내부 조사 함수를 제공.

In [24]:
import inspect
import promotions

ModuleNotFoundError: No module named 'promotions'

In [23]:
# ex 6-8

promos = [func for name, func in inspect.getmembers(promotions, inspect.isfunction)]

def best_promo(order):
    """또 설명"""
    return max(promo(order) for promo in promos)

NameError: name 'promotions' is not defined

- inspect.getmembers() 함수는 조건식으로 걸러낸 객체의 속성들을 반환.
- 모듈 안에서 함수만 걸러내기 위해 insepct.isfunction() 사용

- ex 6-8은 할인 `함수명에 상관없이 작동`.
- 함수들만 promotions 모듈에 넣으면 된다.
- 코드 안에서 암묵적인으로 동의하는 가정 사항.
>???
- 다른 시그너쳐를 가진 함수를 추가하면, order에 적용하는 동안 best_promo() 함수가 오류를 발생시킬 것이다.
> 시그너쳐???<br/>
> 함수의 원형에 명시되는 매개변수 리스트. 어떤 두 함수가 매개변수의 수와 그 타입이 같다면, 두 함수의 시그니처는 같다.

- 객체의 인수를 조사함으로써 더욱 엄격한 테스트 가능.
- 할인 함수르 동적으로 수집할 때, 간단한 데커레이터를 사용하면 더욱 명시적으로 처리할 수 있다. --> 7장에서 하겠다.

## 6.2 명령

- 일반 함수로 대체할 수 있는 단일 메서드 클래스로 구현되는 또 다른 디자인 패턴인 `명령 패턴`에 대해 알아보자.
- 함수를 인수로 전달하는 기법을 사용하면 명령 디자인 패턴도 구현을 단수하게 만들 수 있다.

<img src="image/fig_6-2.JPG">

- 목적 : 연산을 실행하는 객체(호출자 invoker)와 연산을 구현하는 객체(수신자 reveiver)를 분리
- 개본 개념 : 명령 객체를 수신자와 호출자 사이에 놓고, 명령은 execute() 라는 하나의 메서드로 인터페이스를 구현.
- execute() :  원하는 연산을 수행하기 위해 수신자가 가지고 있는 메서드 호출

- 호출자는 수신자의 인터페이스를 알 필요가 없고, 명령의 서브클래스를 통해 서로 다른 수신자를 추가할 수 있다.
- 호출자는 구체적인 명령으로 설정, 연산을 실행하기 위해 execute() 메서드를 호출.
- 그림 6-2에서 MacroCommand 명령 객체는 일련의 명령을 저장. 이때 execute() 메서드는 저장된 각 명령의 execute() 메서드를 호출한다.

- "디자인 패턴"에서는 `명령은 콜백에 대한 객체지향식 대체물`이라고 설명.
- 콜백을 객체지향식 대체물로 바꿀 필요가 있을까? 늘 그런것은 아니다.


- 호출자에 Command 객체 대신 간단히 함수를 바로 지정할 수 있다.
- command.exeute() 대신 command()를 호출하면 된다.


- MacroCommand는 `__call__()` 메서드를 가진 클래스로 구현할 수 있다.
- MacroCommand의 객체는 콜러블이 되며, ex 6-9에서 보는 것처럼 향후에 호출할 함수들의 리스트를 가진다.

In [25]:
# ex 6-9

class MacroCommand:
    """명령 리스트를 실행하는 명령"""
    
    def __init__(self, commands):
        self.commands = list(commands)  # <1> commands 인수로부터 리스트를 만들면 명령들이 반복 가능한 객체임이 보장되며,
                                        # 각각의 MacroCommand객체 안에 명령에 대한 참조를 복사
        
    def __call__(self):
        for command in self.commands:  # <2> MacroCommand 객체가 호출되면 self.commands에 들어 있는 명령이 순서대로 호출된다.
            command()

- 실행 취소 지원 등 공급 기능의 명령 패턴을 구현하려면 단순한 콜백 함수로는 어려울 수도 있지만, 파이썬에서는 두 가지 대안을 제시.
    1. ex 6-9의 MacroCommand와 같은 콜러블 객체는 필요한 상태를 보관함으로써 __call__() 메서드 이외의 메서드도 제공할 수 있다.
    1. 함수가 호출된 후의 상태를 내부에 보관하기 위해 클로저를 사용할 수 있다.

- 명령 패턴을 일급 함수로 대체하는 방법을 생각해보았다.
- 상위 수준에서 보면 이 접근법은 저략패턴에 사용했던 방법과 비슷하다.
- 즉, 단일 메서드 인터페이스를 구현하는 클래스의 객체를 콜러블로 대체하는 것이다.
- 모든 파이썬 콜러블이 `__call__()`이라는 단일 메서드 인터페이스를 구현하므로, 이런 대안이 가능하다.

## 6.3 요약

- "디자인 패턴" 출간 20주년 기념 발표 자료에서, 랄프존슨은 "디자인 패턴"이 '가 디자인 패턴에 대한 단계가 아니라 결과 패턴을 지나치게 강조한 것'이 약점이라고 언급.
- 이 장에서는 먼저 전략 패턴에 대해 일급 함수를 이용해서 단순화할 수 있는 대안을 살펴 보았다.



- 전략이나 명령 패턴을 흉내 내기보다는, 파이썬에서 함수나 콜러블 객체를 이용해서 더욱 자연스럽게 콜백을 구현할 수 있는 경우도 많다.
- 전략 패턴을 리팩토링하고 명령 패턴에 대해 설명하면서 어느 정도 식견을 넓히는 계기가 되었다.
- 일반적인 패턴이나 API는 `일급 함수나 기타 콜러블을 사용해서 파이썬에서 더욱 간결하게 구현할 수 있다`.



- 피터 노빅은 명령 패턴과 전략 패턴은 적어도 어떠한 유스케이스에서는 일급 함수를 이용해서 더욱 간단하게 만들거나 보이지 않게 만들 수 있다는 메시지를 보내고 있다.

## 6.4 읽을거리

- 데커레이터와 클로저는 7장에서.


- "Pyhton Cookbook, 3E"의 8.21절 '비지터 패턴의 구현'에서는 NodeVisitor 클래스가 메서드를 일급 객체로 다루면서 비지터 패턴을 멋지게 구현한다.


- 디자인 패턴에 대한 일반적인 주제를 설명하는 책의 경우 파이썬 프로그래머의 선택 폭은 다른 언어 개발자의 선택 폭보다 좁다.
- 필자가 알기로, 제나디 즐로빈 "Learning Python Design Pattern"이 유일하고, 23가지 중 8개만 설명.

- 타렉 쟈데, "Expert Pyhton Programming" 시중에서 구할 수 있는 최고의 중급 책. 마지막 장에서 고전적인 패턴 7가지를 다룬다.

- 알렉스 마르텔리, EuroPython2011 http://bit.ly/1HGBXvx 에서 발표를 볼 수 있다.

- 브루스 에켈, "파이썬 3 패턴, 비법과 관용구". 2008년부터 쓰기 시작했지만 아직 미완.

- 자바 세계에는 디자인 패턴에 대한 책이 많다. 필자가 좋아하는 책은 에릭 프리먼, 버트 베이츠, 케이시 시에라, 엘리자베스 롭슨의 "Head First Design Patterns". 23개의 패턴 중 16개 설명.

- 덕 타이핑 및 일급 함수를 가진 동적 언어의 관점에서 패턴을 새롲ㅂ게 조명하는 러스 올젠의 "Design Pattern in Guby". 파이썬에도 적용할 수 있는 여러 개념을 소개.

- '동적 언어에서의 디자인 패턴' 슬라이드 http://norvig.com/design-patterns/ 에서 피터 노빅은 일급 함수 및 동적인 특성...

- 물론 "디자인 패턴"이 필독서다.